In [1]:
from __future__ import print_function 

%load_ext autoreload
%autoreload 2

from matplotlib import pyplot as plt
%matplotlib inline

import os, sys
import numpy as np
import time

import tensorflow as tf
from tensorflow.keras import backend as K
K.clear_session()


#reset Keras Session
def reset_keras():
    sess = tf.compat.v1.keras.backend.get_session()
    tf.compat.v1.keras.backend.clear_session()
    sess.close()
    sess = tf.compat.v1.keras.backend.get_session()

    try:
        del classifier # this is from global space - change this as you need
    except:
        pass

    # use the same config as you used to create the session
    config = tf.compat.v1.ConfigProto()
    config.gpu_options.per_process_gpu_memory_fraction = 1
    config.gpu_options.visible_device_list = "0"
    tf.compat.v1.keras.backend.set_session(tf.compat.v1.Session(config=config))

import pandas as pd
import pickle
import gc, re, copy
from sklearn.model_selection import train_test_split
from tensorflow.python.keras.layers import deserialize, serialize
from tensorflow.python.keras.saving import saving_utils

# Project imports 
from data.tasks import *
from data.label_shift import label_shift_linear, plot_labeldist, plot_splitbars
from experiments.models import *
from experiments.training import *
from bounds.bounds import *
from util.batch import *
from util.kl import *
from util.misc import *
reset_keras()


# gpus = tf.config.experimental.list_physical_devices('GPU')
# if gpus:
#   # Restrict TensorFlow to only allocate 1*X GB of memory on the first GPU
#   try:
#     tf.config.experimental.set_virtual_device_configuration(
#         gpus[0],
#         [tf.config.experimental.VirtualDeviceConfiguration(memory_limit=(1024*4))])
#     logical_gpus = tf.config.experimental.list_logical_devices('GPU')
#     print(len(gpus), "Physical GPUs,", len(logical_gpus), "Logical GPUs")
#   except RuntimeError as e:
#     # Virtual devices must be set before GPUs have been initialized
#     print(e)


#project_folder="/cephyr/users/adambre/Alvis/mnist_transfer/"
project_folder="/cephyr/NOBACKUP/groups/snic2021-23-538/mnist_transfer/"

# Load data & Run evaluation 

In [2]:
task = 7
architecture="resnet"
delta=0.05
binary=True
image_size=64
batch_size=32
#epsilons=[0.1]
alphas=[0.3]#0,0.3]
sigmas=[[3,3]]
seed=1
for alpha in alphas:
    print("alpha:"+str(alpha))
    if alpha==0:
        if task==7:
            source_generator, target_generator=load_task(task=task,alpha=alpha,architecture=architecture,binary=True,image_size=image_size,seed=seed)
        else:
            x_source,y_source,x_target,y_target,iw_source, iw_target=load_task(task)
            x_bound=x_source
            y_bound=y_source
    else:
        if task==7:
            prior_generator, bound_generator, target_generator=load_task(task=task,alpha=alpha,architecture=architecture,binary=True,image_size=image_size,seed=seed) ## create bound/prior split here as own generators
        else:
            x_source,y_source,x_target,y_target, iw_source, iw_target=load_task(task)
            x_bound, x_prior, y_bound , y_prior = train_test_split(x_source,y_source,test_size=alpha,random_state=seed)
            iw_bound, iw_prior, _,_ = train_test_split(iw_source,y_source,test_size=alpha,random_state=seed)
            print(iw_target)
#     for epsilon in epsilons:
#         print("epsilon:"+str(epsilon))
    for sigma in sigmas:    
        print("sigma:"+str(sigma))
        paths=posterior_checkpoints(task, alpha, binary=binary,architecture=architecture,seed=seed,image_size=image_size)
        for p in paths:
            if task==7:
                results=compute_bound_parts(task, p, bound_generator=bound_generator, target_generator=target_generator, alpha=alpha, delta=0.05, bound='germain', binary=True, n_classifiers=5, sigma=sigma, seed=seed,batch_size=batch_size,architecture=architecture,image_size=image_size)
            else:
                results=compute_bound_parts(task,p,x_bound=x_bound,y_bound=y_bound,x_target=x_target,y_target=y_target,alpha=alpha,sigma=sigma,seed=seed,binary=binary,architecture=architecture,image_size=image_size,batch_size=batch_size, iw_bound=iw_bound, iw_target=iw_target)
                print(results)
                sys.exit(-1)
            checkpoint = results['checkpoint'].values.ravel()[0]

            if binary:
                result_path=project_folder+"results/"+"task"+str(task)+"/Binary/"+str(architecture)+"/"+str(image_size)+"_"+str(int(100*alpha))+"_"+str(sigma[0])+str(sigma[1])+'_'+str(seed)+'_'+checkpoint+'_results.pkl'
            else:
                result_path=project_folder+"results/"+"task"+str(task)+"/"+str(architecture)+"/"+str(image_size)+"_"+str(int(100*alpha))+\
                "_"+str(sigma[0])+str(sigma[1])+'_'+str(seed)+'_'+checkpoint+'_results.pkl'

            # Create dir
            os.makedirs(os.path.dirname(result_path), exist_ok=True)
            results.to_pickle(result_path)

alpha:0.3
4.039824732229795
3.9663485272338326
length of chestxray in source 22424
amount of chest in source and target 4.0
mean of weights in source, should be 1:  1.0
sigma:[3, 3]
Computing bound components for
   Prior: None
   Posterior: /cephyr/NOBACKUP/groups/snic2021-23-538/mnist_transfer/posteriors/task7/Binary/resnet/64_30_1/1_0.ckpt
Clearing session...
Initializing models...
Loading weights...
Drawing classifiers...
Calculating errors, joint errors and disagreements...
length of predictions 172250
length of predictions 89696
length of predictions 172250
length of weights 172250
mean of weights, should be 1.. 0.9996207146847701
length of predictions 89696
length of weights 89696
mean of weights, should be 1.. 10.973599714591511
length of predictions 172250
length of predictions 89696
length of predictions 172250
length of weights 172250
mean of weights, should be 1.. 0.9996207146847701
length of predictions 89696
length of weights 89696
mean of weights, should be 1.. 10.973599

TypeError: 'int' object is not subscriptable